In [63]:
# !conda create -n crewai python=3.11
# !conda activate crewai -
%pip install -U 'crewai[tools]' mdpdf 
%pip install -U duckduckgo-search
%pip install -U langchain_google_vertexai
!pip install -U langchain-google-vertexai

%pip install --upgrade --quiet  langchain-core langchain-google-vertexai
%pip install mdpdf

!pip install --upgrade google-genai
# gcloud auth application-default login

  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.10.5-py3-none-any.whl.metadata (30 kB)
  Using cached pydantic_core-2.27.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
Using cached pydantic-2.10.5-py3-none-any.whl (431 kB)
Using cached pydantic_core-2.27.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.23.4
    Uninstalling pydantic_core-2.23.4:
      Successfully uninstalled pydantic_core-2.23.4
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.9.2
    Uninstalling pydantic-2.9.2:
      Successfully uninstalled pydantic-2.9.2
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0

In [13]:
import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

LOCATION="us-central1"

FOLDER_NAME="."

# SVC_ACC = os.environ.get("SERVICE_ACCOUNT")
# PROJECT_NUMBER = str(re.search(r'\d+', SVC_ACC).group()) if SVC_ACC else None



In [14]:
from crewai import Agent, Task, Crew, Process
from crewai_tools.tools import FileReadTool
import os, requests, re, mdpdf, subprocess
import uuid

from langchain_community.vectorstores import Chroma
from langchain_community.tools import DuckDuckGoSearchRun

from typing import Dict

from crewai import Agent, Task, Crew, Process
from crewai_tools.tools import FileReadTool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_google_vertexai import ChatVertexAI
from langchain.tools import tool
from vertexai.preview.vision_models import ImageGenerationModel
import mdpdf

In [15]:
### claim_details = # Define story title and description globally


In [64]:
# print(response.candidates[0].content.parts[0].text)

### V2 

In [60]:
def generate(text_input, image_uri, document_uri):
    client = genai.Client(
      vertexai=True,
      project=PROJECT_ID,
      location=LOCATION,
    )

    text1 = types.Part.from_text(text_input)

    image1 = types.Part.from_uri(
      file_uri=image_uri,
      mime_type="image/jpeg",
    )
    document1 = types.Part.from_uri(
      file_uri=document_uri,
      mime_type="application/pdf",
    )

    model = "gemini-2.0-flash-exp"
    contents = [
    types.Content(
      role="user",
      parts=[
        text1,
        image1,
        document1
      ]
    )
    ]
    generate_content_config = types.GenerateContentConfig(
    temperature = 0,
    top_p = 0.95,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",      threshold="OFF"
    )],

    response_text = client.models.generate_content(
        model=model,
        contents=contents,
        config=generate_content_config,
     )
          
    return(response_text)


In [ ]:

# Example Usage (Outside the function):
text_input = """You are a document entity extraction specialist. Given a document, your task is to extract the text value of the following entities:
    {
    "Claim Receipt": {
    "Date Received": null,
    "Time Received": null,
    "Claimant Information": {
      "Name": null,
      "Member ID/Policy Number": null,
      "Date of Birth": null
    },
    "Provider Information": {
      "Provider Name": null,
      "Provider NPI/Tax ID": null
    },
    "Claim Details": {
      "Date(s) of Service": null,
      "Service Description (Brief)": null,
      "Claim Reference Number": null
    },
    "Important Information": null,
    "Contact Information": {
      "Customer Service Phone Number": null,
      "Email Address": null,
      "Website": null
    }
    }
    }

    - The JSON schema must be followed during the extraction.
    - The values must only include text found in the document
    - Do not normalize any entity value.
    - If an entity is not found in the document, set the entity value to null."""
image_uri = "gs://my-project-0004-346516/photo-1686577353812--2023-11-17T09:27:47.430Z.jpg"
document_uri = "gs://avatar-bucket-notebook/prudential-002/Claim Receipt 001.pdf"

response = generate(text_input, image_uri, document_uri)

In [62]:
print(response.candidates[0].content.parts[0].text)

```json
{
"Claim Receipt": {
"Date Received": "2024-02-29",
"Time Received": "10:15 AM",
"Claimant Information": {
  "Name": "Alice M. Smith",
  "Member ID/Policy Number": "ABC1234567",
  "Date of Birth": "1988-07-12"
},
"Provider Information": {
  "Provider Name": "Family Practice Clinic",
  "Provider NPI/Tax ID": "987654321"
},
"Claim Details": {
  "Date(s) of Service": "2024-02-27",
  "Service Description (Brief)": "Doctor's Visit",
  "Claim Reference Number": "CLM-20240229-001"
},
"Important Information": "This receipt confirms that your claim has been received. It is not a guarantee of\ncoverage or payment. Please keep this receipt for your records. You will be notified of\nthe claim status after review. If you have any questions regarding your claim please\ncontact customer service with your Claim Reference Number",
"Contact Information": {
  "Customer Service Phone Number": "555-123-4567",
  "Email Address": "claims@exampleinsurance.com",
  "Website": "www.exampleinsurance.com"


In [65]:

# Example Usage (Outside the function):
text_input = """You are a document entity extraction specialist. Given a document, your task is to extract the text value of the following entities:
    {
    "response": 
    "{"patient_claimant_name": "string", "patient_claimant_birth_date": "string", "patient_claimant_gender": "string", "patient_claimant_address_city": "string", "patient_claimant_address_state": "string", "patient_claimant_address_zip_code": "string", "patient_claimant_phone_number": "string", "patient_claimant_email_address": "string", "member_id_policy_number": "string", "group_number": "string", "primary_insurance_holder_name": "string", "primary_insurance_holder_birth_date": "string", "relationship_to_insured": "string", "patient_received_other_services": "string", "patient_other_insurance": "string", "insurance_company": "string", "policy_number": "string", "group_number_2": "string", "provider_facility_name": "string", "npi_tax_id": "string", "provider_facility_address_city": "string", "provider_facility_address_state": "string", "provider_facility_address_zip_code": "string", "provider_facility_phone_number": "string", "provider_type": "string", "provider_specialty": "string", "service_date_from": "string", "service_date_to": "string", "location_of_service": "string", "description_of_service": "string", "icd_10_diagnosis_code": "string", "cpt_hcpcs_procedure_code": "string", "claim_related_to_accident": "string", "accident_explanation": "string", "service_require_pre_authorization": "string", "authorization_number": "string", "provider_in_network": "string", "signature": "string", "signature_date": "string", "printed_name": "string", "relationship_to_patient": "string"}" 
    }


    - The JSON schema must be followed during the extraction.
    - The values must only include text found in the document
    - Do not normalize any entity value.
    - If an entity is not found in the document, set the entity value to null."""
image_uri = "gs://my-project-0004-346516/photo-1686577353812--2023-11-17T09:27:47.430Z.jpg"
document_uri = "gs://avatar-bucket-notebook/prudential-002/_Claim Form 001.pdf"

response = generate(text_input, image_uri, document_uri)

In [66]:
print(response.candidates[0].content.parts[0].text)

```json
{
"response": {
"patient_claimant_name": "Smith, Alice, M",
"patient_claimant_birth_date": "07/12/1988",
"patient_claimant_gender": "Other",
"patient_claimant_address_city": null,
"patient_claimant_address_state": null,
"patient_claimant_address_zip_code": null,
"patient_claimant_phone_number": null,
"patient_claimant_email_address": null,
"member_id_policy_number": "ABC1234567",
"group_number": null,
"primary_insurance_holder_name": null,
"primary_insurance_holder_birth_date": null,
"relationship_to_insured": null,
"patient_received_other_services": "No",
"patient_other_insurance": "No",
"insurance_company": null,
"policy_number": null,
"group_number_2": null,
"provider_facility_name": "Family Practice Clinic",
"npi_tax_id": "987654321",
"provider_facility_address_city": null,
"provider_facility_address_state": null,
"provider_facility_address_zip_code": null,
"provider_facility_phone_number": null,
"provider_type": "Doctor",
"provider_specialty": null,
"service_date_from": "0

In [ ]:
# Example Usage (Outside the function):
text_input = """You are a document entity extraction specialist. Given a document, your task is to extract the full detailed document:
    {
    "response": "" 
    }

    - The text schema must be followed during the extraction.
    - The values must only include text found in the document
    - Do not normalize any entity value. """

image_uri = "gs://my-project-0004-346516/photo-1686577353812--2023-11-17T09:27:47.430Z.jpg"
document_uri = "gs://avatar-bucket-notebook/prudential-002/Claim Adjurdication FWA - example doc.pdf"

response = generate(text_input, image_uri, document_uri)

In [ ]:
print(response.candidates[0].content.parts[0].text)